# Import Useful Modules 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.grid_search import GridSearchCV as GS

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

from preprocessing_pipeline import preprocessing

D:\Program\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversi

In [2]:
#Determine Model's Save Location

#version=
version="version_6"

In [3]:
if(not(os.path.exists("model/{}".format(version)))):
    os.makedirs("model/{}".format(version))

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [4]:
def read_product_title_data():
    product_title_only=pd.read_csv('data/query.csv',header=None)
    product_title_only["Product Title"]=product_title_only[1]
    product_title_only=product_title_only[["Product Title"]]
    product_title_only.dropna(inplace=True,axis=0)
    
#     return (product_title_only[:300000],
#             product_title_only[300000:600000],
#             product_title_only[600000:900000],
#             product_title_only[900000:1200000],
#             product_title_only[1200000:1500000],
#             product_title_only[1500000:1800000],
#             product_title_only[1800000:])
    
    return product_title_only

In [5]:
data_for_embedding=read_product_title_data()

In [6]:
data_for_embedding.head()

,Product Title
0,gear+camping
1,Lightstick+EXO
2,vivo+v7
3,subwoofer+kolong+aktif
4,Hotpants+anak


# ---------------------------------------------------------------------------------------------------------------

# Construct Word Embedder (Using fasttext)

In [7]:
preprocessor=preprocessing(None,None)

In [8]:
product_title=[preprocessor.remove_parentheses(value) for value in data_for_embedding["Product Title"]]

In [9]:
product_title[:5]

[['gear', 'camping'],
 ['lightstick', 'exo'],
 ['vivo', 'v7'],
 ['subwoofer', 'kolong', 'aktif'],
 ['hotpants', 'anak']]

In [10]:
len(product_title)

2440682

In [32]:
EMBEDDING_DIMENSION=100
EMBEDDING_EPOCH=20
EMBEDDING_WINDOW=7
SEED=291834

#### Train Embedding Model

In [33]:
# membuat model word embedding
print("Start Time : {}\n".format(str(datetime.datetime.now())))
word_embedder = FastText(product_title, size=EMBEDDING_DIMENSION, window=EMBEDDING_WINDOW, min_count=10, workers=4, sg=1, seed=SEED, min_n=5, iter=EMBEDDING_EPOCH,)
print("End Time : {}".format(str(datetime.datetime.now())))

Start Time : 2018-07-12 14:16:46.063211

End Time : 2018-07-12 14:29:13.777870


#### Save Model

In [64]:
#save model
pickle.dump(word_embedder, open("model/{}/word_embedder.pickle".format(version), 'wb'))

# ---------------------------------------------------------------------------------------------------------------

# Word Embedding Evaluation

#### Words With Similar Context Will Have Higher Similarity Degree

In [34]:
word_embedder.wv.most_similar("naruto")

[('20naruto', 0.8509561419487),
 ('boruto', 0.745614230632782),
 ('shippuden', 0.6456469893455505),
 ('tsume', 0.6123440265655518),
 ('kakashi', 0.6100993156433105),
 ('chibi', 0.605150580406189),
 ('rin', 0.6038782596588135),
 ('juggernaut', 0.5976088643074036),
 ('kuroko', 0.5961208939552307),
 ('sao', 0.5954280495643616)]

#### Merk dari barang akan saling berdekatan

In [63]:
word_embedder.wv.most_similar("core")

[('core2', 0.7930219769477844),
 ('i5', 0.7834775447845459),
 ('core2quad', 0.7612477540969849),
 ('i3', 0.7352622747421265),
 ('corei5', 0.7318915724754333),
 ('corei3', 0.7248226404190063),
 ('intel', 0.718172550201416),
 ('i7', 0.7136193513870239),
 ('t4200', 0.7133781909942627),
 ('corei7', 0.7071148157119751)]

### Embedding tidak terlalu terpengaruh typo

In [36]:
word_embedder.wv.most_similar("rhonda")

[('honda', 0.8505690097808838),
 ('20honda', 0.8425036668777466),
 ('zonda', 0.753135621547699),
 ('40kva', 0.7089325189590454),
 ('ronda', 0.7038331031799316),
 ('hd7850', 0.6858668923377991),
 ('evo3', 0.6768960952758789),
 ('20accord', 0.6716963052749634),
 ('kipok', 0.6667416095733643),
 ('honday', 0.6631885766983032)]

In [37]:
word_embedder.wv.most_similar("hondar")

[('honda', 0.8302379846572876),
 ('honday', 0.7343689203262329),
 ('hinda', 0.636542797088623),
 ('nouva', 0.5948721766471863),
 ('civic', 0.5856174230575562),
 ('aspira', 0.5853581428527832),
 ('temlar', 0.5685747861862183),
 ('gtr150', 0.5568150877952576),
 ('1976', 0.5563762187957764),
 ('c70', 0.552899956703186)]

In [38]:
word_embedder.wv.most_similar("samsung")

[('smsung', 0.8695873022079468),
 ('saamsung', 0.8118991255760193),
 ('galaxy', 0.7806167006492615),
 ('20samsung', 0.759710431098938),
 ('samsumg', 0.745133638381958),
 ('sasung', 0.7200661897659302),
 ('20samsunge5', 0.7125160694122314),
 ('samaung', 0.6870971322059631),
 ('smasung', 0.6857773065567017),
 ('galaxs', 0.6605793237686157)]

# ---------------------------------------------------------------------------------------------------------------